In [36]:
import requests
import requests.auth
from requests_oauthlib import OAuth1
import oauth2 as oauth
import time
import os
import json
import pprint
import csv

In [37]:
# Import credentials from local source:
reader = csv.reader(open("/Users/mchifala/Desktop/ATLS_5412/Credentials.csv"))
credentials = {}
for line in reader:
    credentials[line[0]] = line[1]
    

In [38]:
# Get a Reddit credentials:
def getRedditHeaders():
    reddit_id = credentials['reddit_id']
    reddit_secret = credentials['reddit_secret']
    reddit_username = credentials['reddit_username']
    reddit_password = credentials['reddit_password']
    reddit_user_agent = credentials['reddit_user_agent']

    reddit_auth = requests.auth.HTTPBasicAuth(reddit_id, reddit_secret)
    reddit_post_data = {'grant_type': 'password', 'username': reddit_username, 'password': reddit_password}
    reddit_token_headers = {'User-Agent': reddit_user_agent}
    r = requests.post("https://www.reddit.com/api/v1/access_token", auth= reddit_auth, data=reddit_post_data, headers=reddit_token_headers)
    #print(r.json())

    reddit_token = r.json()['token_type'] + ' ' + r.json()['access_token']
    
    # Return header dictionary
    return {"Authorization": reddit_token, "User-Agent": reddit_user_agent, 'Content-Type': 'application/json'}


In [41]:
# Get Twitter credentials:
def getTwitterAuth():
    twitter_key = credentials['twitter_key']
    twitter_secret =  credentials['twitter_secret']
    twitter_token = credentials['twitter_token']
    twitter_secret_token = credentials['twitter_secret_token']
    
    #Return authorization object
    return OAuth1(twitter_key, twitter_secret, twitter_token, twitter_secret_token)

getTwitterAuth()

In [42]:
# Get all available Twitter trends:
twitter_auth = getTwitterAuth()
r = requests.get('https://api.twitter.com/1.1/trends/available.json', auth = twitter_auth)
#print(r.json())

In [43]:
# Extract the location ID codes for trends:
world_codes = []
for i in range(0,len(r.json())):
    world_codes.append(r.json()[i]['woeid'])

test_codes = world_codes[0:10]
print(test_codes)

[1, 2972, 3369, 3444, 3534, 4118, 8676, 8775, 9807, 12723]


In [76]:
set(topics)

{'#4Mar',
 '#ActúaTec2019',
 '#Bournemouth',
 '#BuenLunes',
 '#CafeTacvba',
 '#Carnaval',
 '#DíaDeLaFamilia',
 '#EcuadorEsCarnaval',
 '#FelizLunes',
 '#FinalAmarAMuerte',
 '#GoodVibesConCafé',
 '#GordonBanks',
 '#GuaidoDeRegreso',
 '#IMFC',
 '#IWD2019',
 '#J9CL2019',
 '#LaHuelgaFeministaVa',
 '#MasterChefChile',
 '#MondayMotivation',
 '#NAW2019',
 '#RIPKeithFlint',
 '#RIPLukePerry',
 '#SoompiAwards',
 '#SuperLunes',
 '#TeJuroQueTeAmo',
 '#TeamBTS',
 '#TristeTurno',
 '#TwitterBestFandom',
 '#VamosJuntosALaCalle',
 '#VolvioCartuchos',
 '#afcb',
 '#avfc',
 '#elcaminodelajepes',
 '#guerra',
 '#intrusos2019',
 '#serfieraesunorgullo',
 '#swfc',
 '#الوصل_النصر',
 'AMLO',
 'Absolutely',
 'Alberta',
 'Almada',
 'Anthony Ríos',
 'Argentina',
 'Barranquilla',
 'Bienvenido Presidente',
 'Birmingham',
 'Brad Blair',
 'Charlottetown',
 'Cheers',
 'Dylan McKay',
 'EEUU',
 'Estados Unidos',
 'Everton',
 'FARC',
 'Fine Gael',
 'GRACIAS ELENCO AAM',
 'Gordon Banks OBE',
 'Guanajuato',
 'John Candy',
 'K

In [65]:
#Search each location ID for its specific trends:
topics = []
for code in world_codes:
    r2 = requests.get('https://api.twitter.com/1.1/trends/place.json?id='+str(code), auth = twitter_auth)
    for i in range(0,len(r2.json()[0])):
        print(r2.json()[0]['trends'][i]['name'])
        topics.append((r2.json()[0]['trends'][i]['name']))

Luke Perry
#RIPLukePerry
#الوصل_النصر
Dylan McKay
Luke Perry
#RIPLukePerry
Brad Blair
Dylan McKay
Luke Perry
#RIPLukePerry
#IWD2019
Charlottetown
Luke Perry
#RIPLukePerry
Brad Blair
Dylan McKay
#IMFC
Luke Perry
#RIPLukePerry
Brad Blair
Luke Perry
#RIPLukePerry
Brad Blair
Dylan McKay
Alberta
Luke Perry
#RIPLukePerry
Brad Blair
Luke Perry
#RIPLukePerry
Brad Blair
Dylan McKay
Luke Perry
#MondayMotivation
Keith Flint
John Candy
Liverpool
London
#avfc
Birmingham
Luke Perry
#RIPLukePerry
Keith Flint
#RIPKeithFlint
Keith Flint
Liverpool
#afcb
#Bournemouth
Luke Perry
#RIPLukePerry
Keith Flint
#RIPKeithFlint
Liverpool
Luke Perry
#RIPLukePerry
Keith Flint
The Prodigy
#RIPKeithFlint
Keith Flint
#MondayMotivation
Luke Perry
#RIPLukePerry
Keith Flint
#RIPKeithFlint
#GordonBanks
RIP Keith Flint
#NAW2019
Liverpool
Luke Perry
#RIPLukePerry
Keith Flint
#RIPKeithFlint
Luke Perry
Keith Flint
Prodigy
#RIPKeithFlint
Luke Perry
#RIPLukePerry
Keith Flint
#RIPKeithFlint
Luke Perry
#RIPLukePerry
Keith Flint
#R

KeyError: 0

In [88]:
# Make Reddit queries for trends and upload to Elasticsearch:
reddit_headers = getRedditHeaders()
elastic_headers = {'Content-Type':'application/json'}
for query in set(topics):
    index = 'test_03_04_19_part2'
    r_get = requests.get('https://oauth.reddit.com/r/all/search?q='+query+'&t=day&limit=25', headers = reddit_headers)
    r_get.headers['content-type']
    list = r_get.json()['data']['children']
    print("Remaining Calls:", r_get.headers['x-ratelimit-remaining'])
    print("Time to Reset:", r_get.headers['x-ratelimit-reset'])
    data = []
    fields_to_keep = ['id','title', 'author', 'score', 'subreddit', 'num_comments', 'num_crossposts', 'created_utc']
    for value in list:
        temp = {k: value['data'][k] for k in fields_to_keep}
        temp.update({'hashtag': query})
        data.append({'index': {"_index": index, '_type': '_doc', '_id': 'r_'+ temp['id'] }})
        del temp['id']
        data.append(temp)
        
    if data:
        data_to_post = '\n'.join(json.dumps(d) for d in data) 
        #print(data_to_post)
        #print("\n New topic! \n")
        r_post = requests.post('http://localhost:9200/'+index+'/_bulk', headers = elastic_headers, data=data_to_post+'\n')
        #print(r_post.text)

Remaining Calls: 509.0
Time to Reset: 536
Remaining Calls: 508.0
Time to Reset: 536
Remaining Calls: 507.0
Time to Reset: 536
Remaining Calls: 506.0
Time to Reset: 535
Remaining Calls: 505.0
Time to Reset: 535
Remaining Calls: 504.0
Time to Reset: 535
Remaining Calls: 503.0
Time to Reset: 534
Remaining Calls: 502.0
Time to Reset: 534
Remaining Calls: 501.0
Time to Reset: 534
Remaining Calls: 500.0
Time to Reset: 534
Remaining Calls: 499.0
Time to Reset: 534
Remaining Calls: 498.0
Time to Reset: 533
Remaining Calls: 497.0
Time to Reset: 533
Remaining Calls: 496.0
Time to Reset: 533
Remaining Calls: 495.0
Time to Reset: 533
Remaining Calls: 494.0
Time to Reset: 533
Remaining Calls: 493.0
Time to Reset: 533
Remaining Calls: 492.0
Time to Reset: 532
Remaining Calls: 491.0
Time to Reset: 532
Remaining Calls: 490.0
Time to Reset: 532
Remaining Calls: 489.0
Time to Reset: 532
Remaining Calls: 488.0
Time to Reset: 531
Remaining Calls: 487.0
Time to Reset: 531
Remaining Calls: 486.0
Time to Res

In [13]:
# This is a working example: make sure the indexes match
import json
data = [
    { "index" : { "_index" : "test5", "_type" : "type1", "_id" : "1" } },
    { "field1" : "value1" },
    { "delete" : { "_index" : "test5", "_type" : "type1", "_id" : "2" }, },
    { "create" : { "_index" : "test5", "_type" : "type1", "_id" : "3" }, },
    { "field1" : "value3" },
    { "update" : {"_id" : "1", "_type" : "type1", "_index" : "test5"} },
    { "doc" : {"field2" : "value2"} }
]

data_to_post = '\n'.join(json.dumps(d) for d in data)
print(data_to_post)
#r = requests.post('http://localhost:9200/test5/_bulk', headers = headers, data=data_to_post + '\n')
#print(r.text)

{"index": {"_index": "test5", "_type": "type1", "_id": "1"}}
{"field1": "value1"}
{"delete": {"_index": "test5", "_type": "type1", "_id": "2"}}
{"create": {"_index": "test5", "_type": "type1", "_id": "3"}}
{"field1": "value3"}
{"update": {"_id": "1", "_type": "type1", "_index": "test5"}}
{"doc": {"field2": "value2"}}


In [6]:
# Communicating with Elastic Search:
#r = requests.get('http://localhost:9200/twitter/_doc/TQqw02gBSrHX7MwTUvS-', headers = {'User-agent': 'mchifala'})
requests.post('http://34.73.60.209:9200/hi_yash/_doc/',  json = {"user" : "kimchy",
    "post_date" : "2019-01-28T14:12:12",
    "message" : "Hi Yash"})


<Response [201]>